In [1]:
import numpy as np
import sys, platform
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

print("Python full:", sys.version)          # e.g. '3.12.5 (main, ...)'
print("Python tuple:", sys.version_info)     # e.g. sys.version_info(major=3, minor=12, micro=5, ...)
print("Python short:", platform.python_version())  # e.g. '3.12.5'
print("Executable:", sys.executable)         # path to the kernel's python


Python full: 3.12.9 | packaged by conda-forge | (main, Mar  4 2025, 22:44:42) [Clang 18.1.8 ]
Python tuple: sys.version_info(major=3, minor=12, micro=9, releaselevel='final', serial=0)
Python short: 3.12.9
Executable: /opt/homebrew/anaconda3/envs/qiskit2x/bin/python


In [2]:
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.circuit.library import efficient_su2

service = QiskitRuntimeService()
print('the backends available are: ', service.backends())
backend = service.backends()[0]

n_qubits=16
# circuit = efficient_su2(127, entanglement='linear')
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

param_values = np.random.rand(circuit.num_parameters)
sampler = SamplerV2(mode=backend)
job = sampler.run([(isa_circuit, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

the backends available are:  [<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]
>>> Job ID: d3kme583qtks738biln0
>>> Job Status: QUEUED


In [3]:
print(job.result())

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-10-10 20:05:43', stop='2025-10-10 20:05:45', size=4096>)])}, 'version': 2})


### Try out Sampler options - IBM Runtime Provider

In [4]:
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

service = QiskitRuntimeService()
print('the backends available are: ', service.backends())
backend = service.backends()[0]

n_qubits=16
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

param_values = np.random.rand(circuit.num_parameters)
sampler = SamplerV2(mode=backend)

### set sampler options
sampler.options.default_shots = 2048

job = sampler.run([(isa_circuit, param_values)],shots=4)  # can override defaults when you run it
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
print('results are', job.result())

the backends available are:  [<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]
>>> Job ID: d3kme8pfk6qs73e64udg
>>> Job Status: QUEUED
results are PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-10-10 20:06:12', stop='2025-10-10 20:06:14', size=4>)])}, 'version': 2})


### Dynamical Decoupling

In [5]:
sampler.options.dynamical_decoupling.enable = True

job = sampler.run([(isa_circuit, param_values)],shots=4)  # can override defaults when you run it
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
print('results are', job.result())

>>> Job ID: d3kmedr4kkus739btlhg
>>> Job Status: QUEUED
results are PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-10-10 20:06:17', stop='2025-10-10 20:06:18', size=4>)])}, 'version': 2})


### Twirling

In [6]:
sampler.options.twirling.enable_gates = True
sampler.options.twirling.enable_measure = True
sampler.options.twirling.strategy = "active-accum"      # good default
sampler.options.twirling.num_randomizations = 20        # e.g., 8–32
sampler.options.twirling.shots_per_randomization = "auto"  # splits your shots

job = sampler.run([(isa_circuit, param_values)],shots=4)  # can override defaults when you run it
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
print('results are', job.result())

>>> Job ID: d3kmeepfk6qs73e64ujg
>>> Job Status: QUEUED
results are PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4, num_bits=16>)), metadata={'circuit_metadata': {}, 'num_randomizations': 20})], metadata={'execution': {'execution_spans': ExecutionSpans([TwirledSliceSpanV2(<start='2025-10-10 20:06:22', stop='2025-10-10 20:06:23', size=20>)])}, 'version': 2})


In [7]:
## Error mitigation

In [8]:
### Coding exercise 1
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2, QiskitRuntimeService
from qiskit_aer.primitives import SamplerV2

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)

sv = QiskitRuntimeService()
backend = sv.backends()[0]
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

cir=qc
cir_isa=pm.run(cir)
sampler = SamplerV2(mode=backend, default_shots=3)

job=sampler.run([cir_isa], shots=8000)
results = job.result()

TypeError: SamplerV2.__init__() got an unexpected keyword argument 'default_shots'

In [10]:
### Coding exercise 1
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2, QiskitRuntimeService
from qiskit_aer.primitives import SamplerV2

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.measure_all()

cir=qc
sampler = SamplerV2(default_shots=3)

job=sampler.run([cir], shots=8000)

results = job.result()

In [12]:
results

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=8000, num_bits=2>)), metadata={'shots': 8000, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 1.6458e-05, 'max_memory_mb': 18432, 'time_taken_execute': 0.007812292, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'parallel_experiments': 1}})], metadata={'version': 2})